In [1]:
import arcpy

arcpy.env.workspace = "C:/EsriTraining/firestation.gdb"

print(arcpy.Exists(arcpy.env.workspace))

True


In [6]:
# First check if the workspace/GDB exists

if arcpy.Exists(arcpy.env.workspace):
    # If it exists then list all the feature classes
    feature_classes = arcpy.ListFeatureClasses()
    
    for feature_class in feature_classes:
        print(feature_class)  
    

Environmentally_sensitive_area
School
Existing_fire_station
Sites1


In [26]:
# Clean-up code / function

fcs_to_keep = ["Environmentally_sensitive_area", 
               "School", "Existing_fire_station", "Sites1"]

arcpy.env.workspace = "C:/EsriTraining/firestation.gdb"

if arcpy.Exists(arcpy.env.workspace):
    # If it exists then list all the feature classes
    feature_classes = arcpy.ListFeatureClasses()
    
    for feature_class in feature_classes:
        if feature_class in fcs_to_keep:
            print("Found", feature_class)
        else:
            arcpy.management.Delete(feature_class)
            print("\tDeleting", feature_class)

Found Environmentally_sensitive_area
Found School
Found Existing_fire_station
Found Sites1
	Deleting First_Step


In [7]:
# Convert the code into a function

# call the function like: clean_up_gdb(["Environmentally_sensitive_area", "School", "Existing_fire_station", "Sites1"]) 

def clean_up_gdb(workspace, fcs_to_keep):
    if arcpy.Exists(arcpy.env.workspace):
        # If it exists then list all the feature classes
        feature_classes = arcpy.ListFeatureClasses()

        for feature_class in feature_classes:
            if feature_class in fcs_to_keep:
                print("Found and keeping", feature_class)
            else:
                # Should work for ArcGIS 2.5 as well
                arcpy.management.Delete(feature_class)
                
                print("\tDeleting", feature_class)
                
clean_up_gdb("C:/EsriTraining/firestation.gdb", 
             ["Environmentally_sensitive_area", "School", "Existing_fire_station", "Sites1"]) 

Found and keeping Environmentally_sensitive_area
Found and keeping School
Found and keeping Existing_fire_station
Found and keeping Sites1
	Deleting Environmentally_sensitive_area_Buffer
	Deleting School_Buffer
	Deleting Existing_fire_station_Buffer
	Deleting FirstCriteria_Union
	Deleting FirstCriteria_Dissolve
	Deleting First_Step
	Deleting Final_Output_Lisa


In [13]:
# How to create a new and empty Geodatabase?

root = "C:/EsriTraining"
out_gdb = "output.gdb"

# For ArcGIS Pro 2.7
# How to enable overwriting files
arcpy.env.overwriteOutput = True

if arcpy.Exists(root + "/" + out_gdb):
    arcpy.management.Delete(root + "/" + out_gdb)
    arcpy.CreateFileGDB_management(root, out_gdb)
else:
    arcpy.CreateFileGDB_management(root, out_gdb)

In [22]:
# Print feature class names in firestation.gdb if they have
# a point geometry, conduct a buffer of 2 km on those fcs (w point geometry)
# store the buffer layers in the output.gdb

# arcpy.Buffer_analysis("School", "c:/EsriTraining/output.gdb/School_Buffer", distance)

arcpy.env.workspace = "C:/EsriTraining/firestation.gdb"

def bufferPointsToOutGDB(workspace, out_gdb, distance):
    arcpy.env.workspace = workspace
    feature_classes = arcpy.ListFeatureClasses()
    for feature_class in feature_classes:
        desc = arcpy.Describe(feature_class)
        if desc.shapeType == "Point":
            arcpy.Buffer_analysis(feature_class, out_gdb + "/" + feature_class + "_Buffer", distance)
            print("Creating a", distance, "buffer on", feature_class)
    
# how to call the function?
bufferPointsToOutGDB("C:/EsriTraining/firestation.gdb", "C:/EsriTraining/output.gdb", "2 Kilometers")

Creating a 2 Kilometers buffer on School
Creating a 2 Kilometers buffer on Existing_fire_station


In [43]:
arcpy.env.workspace = "C:/EsriTraining/output.gdb"
feature_classes = arcpy.ListFeatureClasses()
for feature_class in feature_classes:
    print(feature_class)

School
Existing_fire_station
School_Buffer
Existing_fire_station_Buffer
Environmentally_sensitive_area_Buffer_500_Meters
Sites1_Buffer_500_Meters
First_Step_Buffer_500_Meters
Environmentally_sensitive_area_Buffer_1000_Meters
Sites1_Buffer_1000_Meters
First_Step_Buffer_1000_Meters
School_Buffer_1500_Meters
Existing_fire_station_Buffer_1500_Meters
Existing_fire_station_BufferMinRecord


In [27]:
# Write a function that will take in (1) the location of the
# source files (e.g. C:/EsriTraining/firestation.gdb)
# (2) the location of the output GDB 
# (3) type of the geometry for the buffer analysis (e.g. "Polygon" or "Point")
# (4) the distance in number only
# (5) the distance unit
# Add the distance information to the output files
# e.g. if you specify 500 Meters, -> Schools_Buffer_500_Meters

# batchBuffer("C:/EsriTraining/firestation.gdb", "C:/EsriTraining/output.gdb", "Polygon", 500, "Meters")

def batchBuffer(workspace, out_gdb, geom, distance, unit):
    arcpy.env.workspace = workspace
    feature_classes = arcpy.ListFeatureClasses()
    for feature_class in feature_classes:
        desc = arcpy.Describe(feature_class)
        if desc.shapeType == geom:
            # 500, "Meters" -> "500 Meters"
            arcpy.Buffer_analysis(feature_class, out_gdb + "/" + feature_class + 
                                  "_Buffer_" + str(distance) + "_" + unit, 
                                 str(distance) + " " + unit)
            print("Creating a", distance, unit , "buffer on", feature_class)

batchBuffer("C:/EsriTraining/firestation.gdb", "C:/EsriTraining/output.gdb", "Point", 1500, "Meters")

Creating a 1500 Meters buffer on School
Creating a 1500 Meters buffer on Existing_fire_station


In [42]:
# Do the same as above without asking for the geometry type
# Instead, find the feature class with the least number of records / features
# and conduct a 1 km buffer analysis on it
# arcpy.GetCount_management(feature_class)
# arcpy.management.GetCount(feature_class)

my_list = [20, 3, 7, 100]

# How to get the smallest number?

def getMin(a_list):
    min_item = a_list[0]
    for item in a_list:
        if (item < min_item):
            min_item = item
    return min_item

# print(getMin(my_list))

def buffer_on_fc_w_least(workspace, out_gdb):
    arcpy.env.workspace = workspace
    feature_classes = arcpy.ListFeatureClasses()
    fcs_counts = []
    for feature_class in feature_classes:
        # print(arcpy.management.GetCount(feature_class))
        fcs_counts.append([feature_class, arcpy.management.GetCount(feature_class)[0]])
    print(fcs_counts)
    
    min_record = fcs_counts[0]

    for fc_count in fcs_counts:
        print("Comparing", fc_count[1], min_record[1])
        if int(fc_count[1]) < int(min_record[1]):
            print(fc_count[1], "is smaller than", min_record[1])
            min_record = fc_count
    print(min_record)
    
    arcpy.Buffer_analysis(min_record[0], out_gdb + "/" + min_record[0] + "_BufferMinRecord", "1 Kilometers")    
        

buffer_on_fc_w_least("C:/EsriTraining/firestation.gdb", "C:/EsriTraining/output.gdb")

[['Environmentally_sensitive_area', '5'], ['School', '14'], ['Existing_fire_station', '3'], ['Sites1', '15344']]
Comparing 5 5
Comparing 14 5
Comparing 3 5
3 is smaller than 5
Comparing 15344 3
['Existing_fire_station', '3']
